## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

- 단순히 질문이 아닌, 지시사항을 전달해주는 것!


`PromptTemplate`

- 사용자의 **입력 변수**를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다

- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [3]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [5]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [6]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [7]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

- promp 와 LLM을 묶어주는 역할
- chain은 다양한 형태로 나타날 수 있다.

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [9]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI()

chain = prompt | model

In [12]:
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 쉽게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1631546d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x163160410>, root_client=<openai.OpenAI object at 0x14f3a7a50>, root_async_client=<openai.AsyncOpenAI object at 0x163154890>, model_kwargs={}, openai_api_key=SecretStr('**********'))

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [10]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

In [11]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.

chain.invoke(input)

AIMessage(content='인공지능 모델의 학습 원리는 데이터를 입력으로 받아 예측을 수행하는 과정을 말합니다. 이 모델은 입력 데이터와 정답 데이터를 학습하면서 점차적으로 최적의 예측을 할 수 있도록 학습됩니다.\n\n일반적으로, 인공지능 모델은 입력 데이터를 받아 내부적으로 가중치(weight)와 편향(bias)을 조정하면서 학습을 진행합니다. 이러한 가중치와 편향은 입력 데이터를 잘 표현할 수 있는 특징을 학습하고, 최종적으로 정확한 예측을 할 수 있는 모델을 만들어냅니다.\n\n학습 과정은 손실 함수(loss function)를 사용하여 모델이 예측한 값과 실제 정답 값 사이의 차이를 계산하고, 이를 최소화하는 방향으로 모델을 업데이트합니다. 이러한 과정을 여러번 반복하면서 모델은 점차적으로 더 정확한 예측을 할 수 있게 됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 320, 'prompt_tokens': 33, 'total_tokens': 353, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-161f7609-494b-4b69-be50-b7fe0b02fa22-0', usage_metadata={'input_tokens': 33, 'output_tokens': 320, 'total_tokens':

In [13]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 {how} 설명해주세요.")

model = ChatOpenAI()

chain = prompt | model

# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리","how" : "어렵게"}


# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.

chain.invoke(input)

AIMessage(content='인공지능 모델의 학습 원리는 일반적으로 데이터를 입력하고 이를 기반으로 모델이 일정한 패턴이나 규칙을 학습하는 과정을 말합니다. 이러한 과정은 크게 입력 데이터를 처리하고 가중치를 조정하는 단계로 나눌 수 있습니다.\n\n먼저, 입력 데이터는 모델에 주어지고 이를 통해 모델은 일련의 연산을 수행하여 출력을 생성합니다. 이때, 출력과 실제 정답 사이의 오차를 계산하고 이를 최소화하기 위해 모델의 가중치를 조정합니다. 이 과정을 반복하면서 모델은 점차 정확한 패턴이나 규칙을 습득하고 최적의 성능을 발휘할 수 있게 됩니다.\n\n가중치 조정은 주로 경사 하강법이라는 방법을 사용하는데, 이는 모델의 출력과 실제 정답 사이의 오차를 최소화하기 위해 가중치를 조금씩 조정해가는 방식입니다. 경사 하강법은 오차 함수의 기울기를 계산하여 가중치를 업데이트하고, 이를 반복하여 오차를 최소화하는 방향으로 모델을 조정합니다.\n\n이러한 과정을 통해 인공지능 모델은 주어진 데이터로부터 패턴을 학습하고, 이를 통해 새로운 데이터에 대해 예측하거나 분류하는 등의 작업을 수행할 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 450, 'prompt_tokens': 34, 'total_tokens': 484, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'sto

아래는 스트리밍을 출력하는 예시 입니다.

In [ ]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

### 출력파서(Output Parser)


In [15]:
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI()

chain = prompt | model


answer = chain.invoke(input)
answer.content

'인공지능 모델의 학습 원리는 데이터를 사용하여 패턴을 학습하는 과정입니다. 모델은 주어진 데이터를 입력으로 받아서 원하는 결과를 출력하도록 학습됩니다. \n\n이 과정은 크게 입력층, 은닉층, 출력층으로 구성된 인공신경망을 사용하여 이루어집니다. 입력층에서 데이터가 모델로 입력되고, 은닉층에서 데이터를 처리하고 패턴을 학습하며, 출력층에서 모델이 예측한 결과를 출력합니다.\n\n모델은 입력과 출력 사이의 관계를 학습하기 위해 손실 함수를 사용하여 예측 결과와 실제 결과의 차이를 계산하고, 이 손실을 최소화하는 방향으로 가중치를 조정하면서 학습을 진행합니다.\n\n이렇게 모델은 반복적으로 학습 데이터를 통해 오차를 줄여가며 최적의 가중치를 찾아내는 과정을 통해 데이터에 대한 패턴을 학습하고, 새로운 데이터에 대해 정확한 예측을 할 수 있도록 학습됩니다.'

In [17]:
answer.response_metadata

{'token_usage': {'completion_tokens': 343,
  'prompt_tokens': 33,
  'total_tokens': 376,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_name': 'gpt-3.5-turbo-0125',
 'system_fingerprint': None,
 'finish_reason': 'stop',
 'logprobs': None}

In [18]:
from langchain_core.output_parsers import StrOutputParser

# 기본값은 AImessage다
# output값을 str만 반고 싶을때는 stroutputparser를 사용한다.

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [19]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [20]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

'인공지능 모델의 학습 원리는 데이터와 패턴을 학습하여 입력 데이터와 출력 데이터 간의 관계를 파악하는 과정입니다. 이를 위해 모델은 입력 데이터를 받아들여 가중치와 편향을 조절하여 출력 데이터와 비슷한 값을 예측하도록 학습을 진행합니다.\n\n먼저, 모델은 초기에 무작위로 설정된 가중치와 편향을 가지고 있습니다. 이후, 입력 데이터를 모델에 주입하여 예측값을 계산하고, 이를 실제 출력값과 비교합니다. 이 차이를 최소화하는 방향으로 가중치와 편향을 조정하면서 모델이 학습을 진행합니다.\n\n이러한 과정을 여러 번 반복하여 모델이 입력 데이터와 출력 데이터 간의 관계를 더욱 정확하게 파악하고, 새로운 데이터에 대한 예측을 수행할 수 있도록 학습이 이루어집니다. 이렇게 학습된 모델은 새로운 데이터에 대해 정확한 예측을 수행할 수 있게 됩니다.'

In [21]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리는 데이터를 입력으로 받아들이고 이를 통해 패턴을 학습하는 과정을 말합니다. 

먼저, 모델은 주어진 데이터를 입력으로 받아들여 내부의 가중치와 편향을 조정하면서 입력 데이터와 정답 데이터 간의 관계를 학습합니다. 이를 통해 모델은 입력 데이터를 정확하게 분류하거나 예측할 수 있는 능력을 키워나갑니다.

모델은 학습 데이터를 여러 번 반복해서 학습하면서 오차를 최소화하는 방향으로 최적의 가중치와 편향을 찾아나갑니다. 이러한 과정을 통해 모델은 새로운 데이터에 대해 정확한 예측을 할 수 있게 됩니다. 

요약하자면, 인공지능 모델의 학습 원리는 데이터를 입력으로 받아들이고 이를 통해 패턴을 학습하여 새로운 데이터에 대해 예측하는 과정을 말합니다.

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [22]:
a = "안녕하세요 반갑습니다"
a

'안녕하세요 반갑습니다'

In [23]:
a = """안녕하세요
반갑습니다"""
print(a)
# 여러줄을 묶어서 여러줄에 작성하려면 """ 을사용한다.

안녕하세요
반갑습니다


In [ ]:
# {{}}을 통해 중괄호를 2번나타내면 중괄호를 같이 사용한다
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{{question}}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)
prompt

In [24]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='\n당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.\n양식은 [FORMAT]을 참고하여 작성해 주세요.\n\n#상황:\n{question}\n\n#FORMAT:\n- 영어 회화:\n- 한글 해석:\n')

In [25]:

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4-turbo")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [26]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [27]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:
  Customer: Hi, could I see the menu, please?
  Waiter: Of course. Here you go.
  Customer: Thank you. Hmm, what would you recommend?
  Waiter: The grilled salmon is very popular. It comes with steamed vegetables and rice.
  Customer: That sounds good. I'll have the grilled salmon, please.
  Waiter: Would you like something to drink?
  Customer: Yes, could I have a glass of white wine?
  Waiter: Sure, I'll bring that right out with your meal.

- 한글 해석:
  고객: 안녕하세요, 메뉴판 좀 볼 수 있을까요?
  웨이터: 물론입니다. 여기 있어요.
  고객: 고맙습니다. 음, 뭐가 좋을까요?
  웨이터: 그릴에 구운 연어가 인기가 많아요. 찐 채소와 밥이 함께 나옵니다.
  고객: 좋아 보이네요. 그릴에 구운 연어 주세요.
  웨이터: 음료는 필요하신가요?
  고객: 네, 화이트 와인 한 잔 주시겠어요?
  웨이터: 알겠습니다. 식사와 함께 곧 가져다 드리겠습니다.


In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

In [28]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
  - Customer: Hello, I'd like to order a couple of pizzas for delivery, please.
  - Operator: Sure, what type of pizzas would you like?
  - Customer: I'd like one large pepperoni pizza and one medium vegetarian pizza.
  - Operator: Would you like to add any sides or drinks?
  - Customer: Yes, please add two orders of garlic bread and a 2-liter bottle of Coke.
  - Operator: Alright, your total comes to $35.95. Can I have your delivery address?
  - Customer: It's 742 Evergreen Terrace.
  - Operator: Thank you. It will take about 45 minutes for your order to arrive. Will you be paying with cash or card?
  - Customer: I'll pay with card.
  - Operator: Could you please provide the card number?
  - Customer: Sure, it's 1234 5678 9101 1121.
  - Operator: Thank you. Your order is confirmed, and it'll be there in about 45 minutes. Have a great day!

- 한글 해석:
  - 고객: 안녕하세요, 배달로 피자 두 판 주문하고 싶어요.
  - 상담원: 네, 어떤 피자를 드릴까요?
  - 고객: 대형 페페로니 피자 하나와 중형 채식 피자 하나요.
  - 상담원: 사이드 메뉴나 음료도 추가하시겠어요?
 